## <center> Online Retail Preprocessing </center>
---

**Name**: Francisco Aarón Ortega Anguiano

This practice exercise involves preprocessing and exploratory data analysis (EDA) on a dataset sourced from **Kaggle**.

### Dataset columns:
- `InvoiceNo`
- `InvoiceDate`
- `InvoiceTime`
- `StockCode`
- `Description`
- `Quantity`
- `UnitPrice`
- `Totalsale`
- `CustomerID`
- `Country`

#### Import important libraries and read the dataset

In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

retail_df = pd.read_csv('db/Online-Retail.csv')

retail_df.head()

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country
0,536365,01-12-2010,08:26:00 AM,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850.0,United Kingdom
1,536365,01-12-2010,08:26:00 AM,71053,WHITE METAL LANTERN,6,3.39,20.34,17850.0,United Kingdom
2,536365,01-12-2010,08:26:00 AM,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,22.00,17850.0,United Kingdom
3,536365,01-12-2010,08:26:00 AM,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,20.34,17850.0,United Kingdom
4,536365,01-12-2010,08:26:00 AM,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,20.34,17850.0,United Kingdom


In [31]:
print(f"data shape: {retail_df.shape}")

data shape: (541909, 10)


#### Check for duplicates
Our data has duplicates that we are going to drop

In [32]:
print(f"Our data has: {retail_df.duplicated().sum()} duplicates")

Our data has: 5268 duplicates


In [33]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   InvoiceDate  541909 non-null  object 
 2   InvoiceTime  541909 non-null  object 
 3   StockCode    541909 non-null  object 
 4   Description  540455 non-null  object 
 5   Quantity     541909 non-null  int64  
 6   UnitPrice    541909 non-null  float64
 7   Totalsale    541909 non-null  float64
 8   CustomerID   406829 non-null  float64
 9   Country      541909 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 41.3+ MB


#### Check the count of null values per column in the DataFrame
We can drop the missing values in the description, but it has a lot of nulls in CustomerID which isn't very good

In [34]:
retail_df.isnull().sum()

InvoiceNo           0
InvoiceDate         0
InvoiceTime         0
StockCode           0
Description      1454
Quantity            0
UnitPrice           0
Totalsale           0
CustomerID     135080
Country             0
dtype: int64

#### Describe to have a fast overview
We can see that `Quantity`, `UnitPrice` and `Totalsale` have negative values, we are going to explore why.

In [35]:
retail_df.describe()

,Quantity,UnitPrice,Totalsale,CustomerID
count,541909.000000,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,17.987795,15287.690570
std,218.081158,96.759853,378.810824,1713.600303
min,-80995.000000,-11062.060000,-168469.600000,12346.000000
25%,1.000000,1.250000,3.400000,13953.000000
50%,3.000000,2.080000,9.750000,15152.000000
75%,10.000000,4.130000,17.400000,16791.000000
max,80995.000000,38970.000000,168469.600000,18287.000000


I'm going to start by dividing the data into two different dataframes one with the non negative values and the other containing them, to see if the values should be negative or are a mistake that needs to be fixed.

In [36]:
no_neg_values_df = retail_df[
	(retail_df['Quantity'] >= 0) &
	(retail_df['UnitPrice'] >= 0) &
	(retail_df['Totalsale'] >= 0)
]

no_neg_values_df.describe()

,Quantity,UnitPrice,Totalsale,CustomerID
count,531283.000000,531283.000000,531283.000000,397924.000000
mean,10.655299,3.898954,20.077218,15294.315171
std,156.830617,35.876279,270.058254,1713.169877
min,1.000000,0.000000,0.000000,12346.000000
25%,1.000000,1.250000,3.750000,13969.000000
50%,3.000000,2.080000,9.900000,15159.000000
75%,10.000000,4.130000,17.700000,16795.000000
max,80995.000000,13541.330000,168469.600000,18287.000000


In [37]:
neg_values_df = retail_df[
	(retail_df['Quantity'] < 0) |
	(retail_df['UnitPrice'] < 0) |
	(retail_df['Totalsale'] < 0)
]

neg_values_df.describe()

,Quantity,UnitPrice,Totalsale,CustomerID
count,10626.000000,10626.000000,10626.000000,8905.000000
mean,-45.598438,40.217975,-86.480012,14991.667266
std,1092.111602,641.765091,1913.347799,1706.772357
min,-80995.000000,-11062.060000,-168469.600000,12346.000000
25%,-10.000000,1.060000,-15.950000,13510.000000
50%,-2.000000,2.100000,-6.800000,14895.000000
75%,-1.000000,4.950000,-2.100000,16393.000000
max,1.000000,38970.000000,0.000000,18282.000000


In [38]:
neg_values_df.sample(10)

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country
146126,C548972,05-04-2011,11:42:00 AM,21974,SET OF 36 PAISLEY FLOWER DOILIES,-12,1.45,-17.40,15674.0,United Kingdom
326853,C565597,05-09-2011,02:29:00 PM,22980,PANTRY SCRUBBING BRUSH,-1,1.65,-1.65,12709.0,Germany
43783,C540142,05-01-2011,10:50:00 AM,22960,JAM MAKING SET WITH JARS,-12,3.75,-45.00,12782.0,Portugal
365742,C568714,28-09-2011,04:04:00 PM,22775,PURPLE DRAWERKNOB ACRYLIC EDWARDIAN,-8,1.25,-10.00,16332.0,United Kingdom
282904,561668,28-07-2011,05:03:00 PM,44091A,NaN,-2,0.00,0.00,NaN,United Kingdom
361729,C568370,26-09-2011,04:43:00 PM,90199C,5 STRAND GLASS NECKLACE CRYSTAL,-1,6.35,-6.35,15154.0,United Kingdom
394049,C570867,12-10-2011,04:17:00 PM,21976,PACK OF 60 MUSHROOM CAKE CASES,-24,0.55,-13.20,12607.0,USA
148115,549179,06-04-2011,05:34:00 PM,84007,NaN,-3,0.00,0.00,NaN,United Kingdom
28561,C538642,13-12-2010,02:37:00 PM,21658,GLASS BEURRE DISH,-1,3.95,-3.95,12476.0,Germany
44918,C540246,05-01-2011,03:43:00 PM,22201,FRYING PAN BLUE POLKADOT,-1,4.25,-4.25,13081.0,United Kingdom


In [39]:
neg_values_df[(neg_values_df['UnitPrice'] < 0)]

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country
299983,A563186,12-08-2011,02:51:00 PM,B,Adjust bad debt,1,-11062.06,-11062.06,NaN,United Kingdom
299984,A563187,12-08-2011,02:52:00 PM,B,Adjust bad debt,1,-11062.06,-11062.06,NaN,United Kingdom


In [40]:
no_neg_values_df.isnull().sum()

InvoiceNo           0
InvoiceDate         0
InvoiceTime         0
StockCode           0
Description       592
Quantity            0
UnitPrice           0
Totalsale           0
CustomerID     133359
Country             0
dtype: int64

In [41]:
neg_values_df.isnull().sum()

InvoiceNo         0
InvoiceDate       0
InvoiceTime       0
StockCode         0
Description     862
Quantity          0
UnitPrice         0
Totalsale         0
CustomerID     1721
Country           0
dtype: int64

In [42]:
print(f"no negative values data shape: {no_neg_values_df.shape}")
print(f"negative values data shape: {neg_values_df.shape}")

no negative values data shape: (531283, 10)
negative values data shape: (10626, 10)


It seems like the columns `Quantity`, `UnitPrice` and `Totalsale` have negative for different motives related to the business, so for now we are going to focus exclusively in the data that has no negatives in it. We can see that most of the null values are from this part of the data, which isn't surprisely seen that most of the data are non negative.

---

#### Clean the data

- Drop duplicates
- Drop all the values in `Description`
- `CustomerID` has almost 140k missing values, almost 25% of the data
- `Quantity`, `UnitPrice` and `Totalsale` having negatives can be easily solve using absolute numbers

In [43]:
# make a copy of the non negative df
retail_df2 = no_neg_values_df.copy()

In [44]:
# dropping all duplicates
retail_df2.drop_duplicates(inplace=True)

# resetting the index
retail_df2.reset_index(drop=True, inplace=True)

In [45]:
# dropping missing values
retail_df2.dropna(subset=['Description'], inplace=True)

# making all items lower
retail_df2['Description'] = retail_df2['Description'].apply(str.lower)

# remove leading and trailing white spaces, and removing "."
retail_df2['Description'] = retail_df2['Description'].apply(lambda x: x.strip().replace(".", ""))

In [46]:
# replace the missing values to 0
retail_df2['CustomerID'] = retail_df2['CustomerID'].fillna(0)

# another solution is to drop the rows with missing CustomerID, however in this case it's less recommended for the kind of analysis that I will be doing since nearly 140k rows would be lost, resulting in a significant loss of information.
# retail_df2.dropna(subset=['CustomerID'], inplace=True)

In [47]:
retail_df2.isnull().sum()

InvoiceNo      0
InvoiceDate    0
InvoiceTime    0
StockCode      0
Description    0
Quantity       0
UnitPrice      0
Totalsale      0
CustomerID     0
Country        0
dtype: int64

#### Simple Feature Engineering

In [48]:
# making sure all the invoice dates are formatted well
retail_df2['InvoiceDate'] = pd.to_datetime(retail_df2['InvoiceDate'], format="%d-%m-%Y")

In [49]:
# extracting Year and Month columns
retail_df2['Year'] = pd.DatetimeIndex(retail_df2['InvoiceDate']).year
retail_df2['Month'] = pd.DatetimeIndex(retail_df2['InvoiceDate']).month

In [50]:
# making seasonal column
month_to_season = {
	1: "Winter",
	2: "Winter",
	3: "Spring",
	4: "Spring",
	5: "Spring",
	6: "Summer",
	7: "Summer",
	8: "Summer",
	9: "Fall",
	10: "Fall",
	11: "Fall",
	12: "Winter"
}

retail_df2['Season'] = retail_df2['Month'].map(month_to_season)

In [51]:
retail_df2[['InvoiceDate', 'Year', 'Month', 'Season']].sample(5)

,InvoiceDate,Year,Month,Season
328800,2011-09-13,2011,9,Fall
434204,2011-11-08,2011,11,Fall
424805,2011-11-03,2011,11,Fall
516561,2011-12-06,2011,12,Winter
47947,2011-01-09,2011,1,Winter


In [52]:
def extract_am_pm(time):
	"""
	Takes the time as a string and extracts if it's AM or PM

	Args:
		time (str): time as a string
	return:
		AM or PM
	"""
	# Getting the part after the " " of the string time
	am_pm = time.split()[1]

	return am_pm

In [53]:
retail_df2['AM_or_PM'] = retail_df2['InvoiceTime'].apply(extract_am_pm)

In [54]:
retail_df2[['InvoiceTime', 'AM_or_PM']].sample(5)

,InvoiceTime,AM_or_PM
450254,04:16:00 PM,PM
457634,10:01:00 AM,AM
525744,11:20:00 AM,AM
257786,04:27:00 PM,PM
56756,11:08:00 AM,AM


In [55]:
retail_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 525460 entries, 0 to 526051
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    525460 non-null  object        
 1   InvoiceDate  525460 non-null  datetime64[ns]
 2   InvoiceTime  525460 non-null  object        
 3   StockCode    525460 non-null  object        
 4   Description  525460 non-null  object        
 5   Quantity     525460 non-null  int64         
 6   UnitPrice    525460 non-null  float64       
 7   Totalsale    525460 non-null  float64       
 8   CustomerID   525460 non-null  float64       
 9   Country      525460 non-null  object        
 10  Year         525460 non-null  int32         
 11  Month        525460 non-null  int32         
 12  Season       525460 non-null  object        
 13  AM_or_PM     525460 non-null  object        
dtypes: datetime64[ns](1), float64(3), int32(2), int64(1), object(7)
memory usage: 56.1+ MB


In [56]:
# save the new transformed data into a csv
retail_df2.to_csv('db/Online-Retail-Transformed.csv', index=False)

---
### Next part of the project
Once finished with the preprocessing of the data with can start a EDA

[2. Exploratory Data Analysis Univariate](2.online_retail_univariate.ipynb)